# **Database Creation:**

The focus of this notebook is to create the database that will be used (imported) at the start of the data exploration notebook. This will create an esacstive batabase that is close to 12GB and ~45 million rows. Below I have code that will pull information from the NYC OPEN data's API. When Saving this document I will removing my username and password, to be able to execute the code below an account needs to be created [here](https://data.cityofnewyork.us/login). Then enter your username, password, and token into the client variable below.

Below the API pulling code I will be using a more repeatable method and just reading the csv files of the data bases. The API pulling will take a few hours so this saves me a lot of time when needing to go back through this section.

In [1]:
import pandas as pd
from sodapy import Socrata
from tenacity import retry, stop_after_attempt, wait_fixed, RetryError
import time
from datetime import timedelta

In [ ]:
# HERE is where to enter the NYC open data credentials
client = Socrata('data.cityofnewyork.us',
                 '2zEO885pCsTLIk5R1E9lnRTm6', #Token: str
                 username="Cmphillipps73@gmail.com",
                 password="Roxz124!",
                 timeout=60)

In [ ]:
#request to pull from API function
@retry(stop=stop_after_attempt(5), wait=wait_fixed(10))
def call_chunk(client, dataset_id, chunk_size, offset):
    return client.get(dataset_id, limit=chunk_size, offset=offset, order="cad_evnt_id ASC")


#Full Pull chunk from API function
def pull_chunk(dataset_id, client, entry=0, chunk_size=100000):
    
    data_frame = []  # Empty dataframe to append pulls to
    offset = entry  # Initial entry point (default is 0)
    
    # Start time
    start_time = time.time()
    time_limit = timedelta(minutes=60).total_seconds()  # 100 minutes in seconds

    while True:
        # Check if the time limit has been reached
        elapsed_time = time.time() - start_time
        if elapsed_time > time_limit:
            print(f"Time limit reached. Ending data pull at offset {offset}.")
            break

        try:
            # Fetch data with retry logic
            results = call_chunk(client, dataset_id, chunk_size, offset)
            results_df = pd.DataFrame.from_records(results)

            # Break the loop if no more data is returned
            if results_df.empty:
                break

            # Append the fetched data to the list of data frames
            data_frame.append(results_df)

            # Increment the offset
            offset += chunk_size

        except RetryError:
            print(f"Failed to fetch data after several retries at offset {offset}.")
            break

    # Concatenate all data frames into a single data frame
    df = pd.concat(data_frame, ignore_index=True)
    return df

We are going to pull all the data from the NYPD calls for service historic dataset and save it to `historic_df`. We may have to do this a couple of times and concat the results together to be sure we are pulling all the data. We will know that we have pulled all the data as the starting ID is 43,928,145 and the ending ID is 99,844,062. If we see that the ending ID is not equal to 99,844,062 the we know we need to run the function again with the ending offset as the entry point.

In [ ]:
#Historic pull 1

#dataset id referencing the historic dataset
dataset_id = "d6zx-ckhd"

historic1_df = pull_chunk(dataset_id, client)

#check to see how it did
display(historic1_df.head())
display(historic1_df.tail())

In [ ]:
#Exporting to csv to save progress
pd.DataFrame.to_csv(historic1_df, r'C:\Users\cmphi\Documents\BrainStation\DataBases\historic1_df.csv')

This is one run and will most likely not pull all the data. So iterate by updating the entry with the ending offset and save the next pull the `historic2_df`. Then mash all the pulls together to create a `final_historic_df` using the code below.

```python
# stack the four DataFrames
final_historic_df = pd.concat([historic1_df, historic2_df, historic3_df, ignore_index=True,axis=0))
```

### **YTD:**

Now it is time to pull the year to data dataset. This is a much smaller dataset and can probably be done in one pull, however if not then repeat the steps above to create multiple pulls for ytd then concat them together.

In [ ]:
dataset_id = "n2zq-pubd"

ytd_df = pull_chunk(dataset_id, client)


#check to see how it did
display(ytd_df.head())
display(ytd_df.tail())

In [ ]:
pd.DataFrame.to_csv(ytd_df, r'C:\Users\cmphi\Documents\BrainStation\DataBases\ytd_df.csv')

# **Simple read csv impport:**

So the code above is all to pull the data from open data's api. But below I will be using a short cut and importing from a local file.

In [2]:
historic_df = pd.read_csv(r"C:\Users\cmphi\Documents\BrainStation\DataBases\capstone_911\NYPD_Calls_for_Service__Historic__20240613.csv")
historic_df.info()

C:\Users\cmphi\AppData\Local\Temp\ipykernel_4732\3304922866.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  historic_df = pd.read_csv(r"C:\Users\cmphi\Documents\BrainStation\DataBases\capstone_911\NYPD_Calls_for_Service__Historic__20240613.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40667946 entries, 0 to 40667945
Data columns (total 20 columns):
 #   Column         Dtype  
---  ------         -----  
 0   OBJECTID       float64
 1   CAD_EVNT_ID    int64  
 2   CREATE_DATE    object 
 3   INCIDENT_DATE  object 
 4   INCIDENT_TIME  object 
 5   NYPD_PCT_CD    float64
 6   BORO_NM        object 
 7   PATRL_BORO_NM  object 
 8   GEO_CD_X       int64  
 9   GEO_CD_Y       int64  
 10  RADIO_CODE     object 
 11  TYP_DESC       object 
 12  CIP_JOBS       object 
 13  ADD_TS         object 
 14  DISP_TS        object 
 15  ARRIVD_TS      object 
 16  CLOSNG_TS      object 
 17  Latitude       float64
 18  Longitude      float64
 19  Location       object 
dtypes: float64(4), int64(3), object(13)
memory usage: 6.1+ GB


In [3]:
ytd_df = pd.read_csv(r"C:\Users\cmphi\Documents\BrainStation\DataBases\capstone_911\NYPD_Calls_for_Service__Year_to_Date__20240520.csv")
ytd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1805313 entries, 0 to 1805312
Data columns (total 18 columns):
 #   Column         Dtype  
---  ------         -----  
 0   CAD_EVNT_ID    int64  
 1   CREATE_DATE    object 
 2   INCIDENT_DATE  object 
 3   INCIDENT_TIME  object 
 4   NYPD_PCT_CD    int64  
 5   BORO_NM        object 
 6   PATRL_BORO_NM  object 
 7   GEO_CD_X       int64  
 8   GEO_CD_Y       int64  
 9   RADIO_CODE     object 
 10  TYP_DESC       object 
 11  CIP_JOBS       object 
 12  ADD_TS         object 
 13  DISP_TS        object 
 14  ARRIVD_TS      object 
 15  CLOSNG_TS      object 
 16  Latitude       float64
 17  Longitude      float64
dtypes: float64(2), int64(4), object(12)
memory usage: 247.9+ MB


In [4]:
#Mashing the two dataframes together 
api_df = pd.concat([historic_df, ytd_df], ignore_index=True, axis=0)

In [5]:
#Taking a peak at what the merged dataframes looks like
display(api_df.info())
display(api_df.head())
display(api_df.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42473259 entries, 0 to 42473258
Data columns (total 20 columns):
 #   Column         Dtype  
---  ------         -----  
 0   OBJECTID       float64
 1   CAD_EVNT_ID    int64  
 2   CREATE_DATE    object 
 3   INCIDENT_DATE  object 
 4   INCIDENT_TIME  object 
 5   NYPD_PCT_CD    float64
 6   BORO_NM        object 
 7   PATRL_BORO_NM  object 
 8   GEO_CD_X       int64  
 9   GEO_CD_Y       int64  
 10  RADIO_CODE     object 
 11  TYP_DESC       object 
 12  CIP_JOBS       object 
 13  ADD_TS         object 
 14  DISP_TS        object 
 15  ARRIVD_TS      object 
 16  CLOSNG_TS      object 
 17  Latitude       float64
 18  Longitude      float64
 19  Location       object 
dtypes: float64(4), int64(3), object(13)
memory usage: 6.3+ GB


None

,OBJECTID,CAD_EVNT_ID,CREATE_DATE,INCIDENT_DATE,INCIDENT_TIME,NYPD_PCT_CD,BORO_NM,PATRL_BORO_NM,GEO_CD_X,GEO_CD_Y,RADIO_CODE,TYP_DESC,CIP_JOBS,ADD_TS,DISP_TS,ARRIVD_TS,CLOSNG_TS,Latitude,Longitude,Location
0,4681449.0,67676761,04/06/2020,04/06/2020,1899-12-30T00:00:00.000,49.0,BRONX,PATROL BORO BRONX,1023609,255034,39T2,OTHER CRIMES (IN PROGRESS): TRESPASS/OUTSIDE,Non Critical,04/06/2020 05:20:22 PM,04/06/2020 05:23:40 PM,04/06/2020 05:26:12 PM,04/06/2020 05:31:15 PM,40.866605,-73.857699,POINT (-73.8576994 40.866604513)
1,2409995.0,69112593,05/27/2020,05/27/2020,1899-12-30T00:00:00.000,6.0,MANHATTAN,PATROL BORO MAN SOUTH,984016,205081,75D,VISIBILITY PATROL: DIRECTED,Non CIP,05/27/2020 12:57:11 AM,05/27/2020 12:57:11 AM,05/27/2020 12:57:12 AM,05/27/2020 01:31:10 AM,40.729585,-74.000846,POINT (-74.000845732 40.729584871)
2,4680908.0,67555649,04/02/2020,04/02/2020,1899-12-30T00:00:00.000,40.0,BRONX,PATROL BORO BRONX,1009769,236748,39V6,OTHER CRIMES (IN PROGRESS): VIOL ORDER PROTECT...,Non Critical,04/02/2020 09:33:27 AM,04/02/2020 09:37:42 AM,04/02/2020 09:38:11 AM,04/02/2020 09:49:08 AM,40.816466,-73.907807,POINT (-73.907807308 40.816465657)
3,4662784.0,68094208,04/22/2020,04/22/2020,1899-12-30T00:00:00.000,121.0,STATEN ISLAND,PATROL BORO STATEN ISLAND,940615,152713,10V2,INVESTIGATE/POSSIBLE CRIME: SUSP VEHICLE/OUTSIDE,Non CIP,04/22/2020 03:22:42 AM,04/22/2020 03:22:48 AM,04/22/2020 03:22:56 AM,04/22/2020 03:23:02 AM,40.585739,-74.157100,POINT (-74.15709984 40.585738816)
4,2406519.0,65620161,01/27/2020,01/27/2020,1899-12-30T00:00:00.000,7.0,MANHATTAN,PATROL BORO MAN SOUTH,986654,199390,75D,VISIBILITY PATROL: DIRECTED,Non CIP,01/27/2020 03:55:36 PM,01/27/2020 03:55:37 PM,01/27/2020 03:55:40 PM,01/27/2020 05:50:17 PM,40.713964,-73.991330,POINT (-73.991329718 40.713964124)


,OBJECTID,CAD_EVNT_ID,CREATE_DATE,INCIDENT_DATE,INCIDENT_TIME,NYPD_PCT_CD,BORO_NM,PATRL_BORO_NM,GEO_CD_X,GEO_CD_Y,RADIO_CODE,TYP_DESC,CIP_JOBS,ADD_TS,DISP_TS,ARRIVD_TS,CLOSNG_TS,Latitude,Longitude,Location
42473254,NaN,102036070,03/31/2024,03/31/2024,NaN,73.0,BROOKLYN,PATROL BORO BKLYN NORTH,1007924,184019,68Q1,SEE COMPLAINANT: OTHER/INSIDE,Non CIP,03/31/2024 11:59:39 PM,04/01/2024 12:05:31 AM,04/01/2024 12:28:45 AM,04/01/2024 01:10:38 AM,40.671743,-73.914658,NaN
42473255,NaN,102036071,03/31/2024,03/31/2024,NaN,25.0,MANHATTAN,PATROL BORO MAN NORTH,1001797,232216,75S,STATION INSPECTION BY TRANSIT BUREAU PERSONNEL,Non CIP,03/31/2024 11:59:47 PM,03/31/2024 11:59:47 PM,03/31/2024 11:59:47 PM,04/01/2024 01:12:08 AM,40.804046,-73.936620,NaN
42473256,NaN,102036072,03/31/2024,03/31/2024,NaN,106.0,QUEENS,PATROL BORO QUEENS SOUTH,1023716,186615,75S,STATION INSPECTION BY TRANSIT BUREAU PERSONNEL,Non CIP,03/31/2024 11:59:48 PM,03/31/2024 11:59:48 PM,03/31/2024 11:59:48 PM,04/01/2024 01:13:51 AM,40.678812,-73.857714,NaN
42473257,NaN,102036073,03/31/2024,03/31/2024,NaN,60.0,BROOKLYN,PATROL BORO BKLYN SOUTH,984688,150503,75I,VISIBILITY PATROL: INTERIOR,Non CIP,03/31/2024 11:59:52 PM,03/31/2024 11:59:52 PM,03/31/2024 11:59:52 PM,04/01/2024 12:43:29 AM,40.579780,-73.998425,NaN
42473258,NaN,102036074,03/31/2024,03/31/2024,NaN,46.0,BRONX,PATROL BORO BRONX,1008465,248359,75M,TRAIN RUN/MOBILE ORDER MAINTENANCE SWEEP,Non CIP,03/31/2024 11:59:56 PM,03/31/2024 11:59:57 PM,03/31/2024 11:59:57 PM,04/01/2024 12:05:15 AM,40.848338,-73.912476,NaN


In [6]:
#pushing final df to csv
pd.DataFrame.to_csv(api_df, r'C:\Users\cmphi\Documents\BrainStation\DataBases\capstone_911\api_df.csv', index=False)